# DPO Fine-Tuning: Preference-Tuned Summarizer

In [1]:
# Install dependencies (correct dev version of trl)
!pip install git+https://github.com/huggingface/trl@main
!pip install -q transformers datasets accelerate

  Cloning https://github.com/huggingface/trl (to revision main) to /tmp/pip-req-build-h2a6y2qp
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/trl /tmp/pip-req-build-h2a6y2qp
  Resolved https://github.com/huggingface/trl to commit d98d53983b6cf6b3381cb084c75b93d8e1ba4c52
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:

In [ ]:
# Upload DPO dataset (JSON format with prompt, chosen, rejected)
from google.colab import files
uploaded = files.upload()  # Upload `dpo_format.json`

Saving dpo_format.json to dpo_format.json


In [3]:
# DPO Training Script (DistilGPT2)
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import DPOTrainer, DPOConfig
import torch

In [ ]:
# Load tokenizer and base model
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Ensure padding

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [5]:
model = AutoModelForCausalLM.from_pretrained(model_name)
ref_model = AutoModelForCausalLM.from_pretrained(model_name)

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [6]:
# Load dataset
dataset = load_dataset("json", data_files="dpo_format.json", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# with open("configs/dpo_config.json", "r") as f:
#     config_dict = json.load(f)

# training_args = DPOConfig(**config_dict)

In [7]:
# Define DPO config
training_args = DPOConfig(
    output_dir="./distilgpt2-dpo-checkpoint",
    per_device_train_batch_size=4,
    learning_rate=5e-5,
    num_train_epochs=3,
    logging_dir="./logs",
    save_strategy="epoch",
    save_total_limit=1,  # only the latest checkpoint
    bf16=False,
    fp16=True,  # Enable for Colab GPU
    remove_unused_columns=False,
    report_to="none",
    padding_value=tokenizer.pad_token_id,
)

In [8]:
# Initialize trainer
trainer = DPOTrainer(
    model=model,
    ref_model=ref_model,
    args=training_args,
    train_dataset=dataset,
)

Extracting prompt in train dataset:   0%|          | 0/2871 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/2871 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2871 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1198 > 1024). Running this sequence through the model will result in indexing errors


In [9]:
# Train
trainer.train()

Step,Training Loss
10,0.453700
20,0.052400
30,0.022500
40,0.006600
50,0.006400
60,0.002900
70,0.011200
80,0.003200
90,0.002200
100,0.001000


TrainOutput(global_step=2154, training_loss=0.0029919418935477433, metrics={'train_runtime': 1067.5043, 'train_samples_per_second': 8.068, 'train_steps_per_second': 2.018, 'total_flos': 0.0, 'train_loss': 0.0029919418935477433, 'epoch': 3.0})

In [10]:
# Save final model
trainer.save_model("./distilgpt2-dpo-checkpoint")

In [11]:
# Step 1: Zip the folder
!zip -r distilgpt2-dpo-checkpoint.zip distilgpt2-dpo-checkpoint/

# Step 2: Download the zip file
from google.colab import files
files.download("distilgpt2-dpo-checkpoint.zip")

  adding: distilgpt2-dpo-checkpoint/ (stored 0%)
  adding: distilgpt2-dpo-checkpoint/vocab.json (deflated 59%)
  adding: distilgpt2-dpo-checkpoint/merges.txt (deflated 53%)
  adding: distilgpt2-dpo-checkpoint/README.md (deflated 47%)
  adding: distilgpt2-dpo-checkpoint/special_tokens_map.json (deflated 52%)
  adding: distilgpt2-dpo-checkpoint/training_args.bin (deflated 52%)
  adding: distilgpt2-dpo-checkpoint/tokenizer.json (deflated 82%)
  adding: distilgpt2-dpo-checkpoint/model.safetensors (deflated 7%)
  adding: distilgpt2-dpo-checkpoint/tokenizer_config.json (deflated 52%)
  adding: distilgpt2-dpo-checkpoint/config.json (deflated 52%)
  adding: distilgpt2-dpo-checkpoint/checkpoint-2154/ (stored 0%)
  adding: distilgpt2-dpo-checkpoint/checkpoint-2154/vocab.json (deflated 59%)
  adding: distilgpt2-dpo-checkpoint/checkpoint-2154/merges.txt (deflated 53%)
  adding: distilgpt2-dpo-checkpoint/checkpoint-2154/rng_state.pth (deflated 25%)
  adding: distilgpt2-dpo-checkpoint/checkpoint-215

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>